### sparse Autoencoder for states
#### code adapted from darkefyre

In [1]:
import tensorflow as tf
import keras
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import pickle
import math
import copy
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [15]:
state_features = ['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission']
print (state_features)

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission']


In [73]:
train_df = pd.read_csv('../../data/train_scaled.csv')

In [74]:
train_df.head()

,bloc,icustayid,gender,age,elixhauser,re_admission,SOFA,SIRS,Weight_kg,GCS,...,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev,sedation,mechvent,rrt,died_in_hosp,mortality_90d,reward
0,0.000000,12.0,1.0,0.203388,0.000000,0.0,0.304348,0.25,0.183842,1.000000,...,0.000000,0.000000,0.000000,0.177479,0.0,0.0,0.0,0.0,0.0,0.0
1,0.222560,12.0,1.0,0.203388,0.000000,0.0,0.130435,0.50,0.183842,1.000000,...,0.000000,0.616278,0.782139,0.175772,0.0,0.0,0.0,0.0,0.0,0.0
2,0.356608,12.0,1.0,0.203388,0.000000,0.0,0.086957,0.50,0.183842,1.000000,...,0.000000,0.659309,0.767260,0.174328,0.0,0.0,0.0,0.0,0.0,0.0
3,0.452837,12.0,1.0,0.203388,0.000000,0.0,0.217391,0.50,0.183842,1.000000,...,0.000000,0.659309,0.000000,0.174328,0.0,0.0,0.0,0.0,0.0,15.0
4,0.000000,14.0,0.0,0.909010,0.142857,0.0,0.217391,0.50,0.204211,0.047619,...,0.822795,0.570801,0.657302,0.179999,1.0,1.0,0.0,0.0,1.0,0.0


In [5]:
keys = train_df.keys()

In [75]:
test_df = pd.read_csv('../../data/test_scaled.csv')

In [76]:
test_df.head()

,bloc,icustayid,gender,age,elixhauser,re_admission,SOFA,SIRS,Weight_kg,GCS,...,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev,sedation,mechvent,rrt,died_in_hosp,mortality_90d,reward
0,0.000000,61.0,0.0,0.3779,0.230769,0.0,0.545455,0.75,0.926956,0.40625,...,0.802778,0.000000,0.000000,0.601591,0.0,0.0,0.0,0.0,0.0,0.0
1,0.222560,61.0,0.0,0.3779,0.230769,0.0,0.363636,0.75,0.926956,0.37500,...,0.867793,0.565062,0.747928,0.607028,0.0,0.0,0.0,0.0,0.0,0.0
2,0.356608,61.0,0.0,0.3779,0.230769,0.0,0.363636,1.00,0.926956,0.34375,...,0.844265,0.600596,0.713579,0.611279,0.0,0.0,0.0,0.0,0.0,0.0
3,0.452837,61.0,0.0,0.3779,0.230769,0.0,0.318182,1.00,0.926956,0.31250,...,0.845492,0.623478,0.711891,0.615618,0.0,0.0,0.0,0.0,0.0,0.0
4,0.527957,61.0,0.0,0.3779,0.230769,0.0,0.318182,1.00,0.926956,0.28125,...,0.846569,0.643104,0.726051,0.619879,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
X_train, X_val, _, _ = train_test_split(train_df, train_df, test_size=0.2, random_state=42)

In [78]:
training_df = pd.DataFrame(X_train, columns=keys)
validating_df = pd.DataFrame(X_val, columns=keys)

In [79]:
training_df.sample(2).loc[:,state_features].head()

,Albumin,Arterial_BE,Arterial_lactate,Arterial_pH,BUN,CO2_mEqL,Calcium,Chloride,Creatinine,DiaBP,...,Weight_kg,age,elixhauser,gender,mechvent,output_4hourly,output_total,paCO2,paO2,re_admission
38817,0.319149,0.328467,0.045253,0.578947,0.549569,0.148980,0.366492,0.651316,0.333683,0.501647,...,0.216517,0.706404,0.285714,1.0,0.0,0.67717,0.549225,0.110917,0.164029,0.0
64199,0.255319,0.381022,0.074842,0.562105,0.592486,0.194898,0.459686,0.688947,0.305758,0.590588,...,0.203950,0.954007,0.285714,1.0,1.0,0.00000,0.606318,0.169294,0.207194,0.0


In [20]:
n_input = len(state_features)
n_hidden_1 = 200
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.zeros([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.zeros([n_input])),
}

In [66]:
class AutoEncoder:
    def __init__(self):
        self.input_size = n_input
        self.batch_size = tf.placeholder(tf.int32)
        self.sparsity_dist = tf.fill([self.batch_size, n_hidden_1], 0.05)
        self.X = tf.placeholder("float", [None, self.input_size])
        self.layer_1_a = tf.nn.sigmoid(tf.add(tf.matmul(self.X, weights['encoder_h1']),
                                   biases['encoder_b1']))
        self.layer_1_d = tf.nn.sigmoid(tf.add(tf.matmul(self.layer_1_a, weights['decoder_h1']),
                                   biases['decoder_b1']))
        self.target_y = self.X
        self.kl = tf.reduce_mean(tf.reduce_sum(tf.multiply(self.sparsity_dist, tf.log(self.sparsity_dist/self.layer_1_a)) + \
                                   tf.multiply((1-self.sparsity_dist),tf.log((1-self.sparsity_dist)/(1-self.layer_1_a))), axis =1))
        #self.kl = tf.reduce_mean(tf.contrib.keras.losses.kullback_leibler_divergence(self.sparsity_dist, self.layer_1_a))  
    # regularisation term logic - loss at the start is roughly 60, of this 0.5 is reconstruction loss.  
        self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2)) + 0.0001 * self.kl
        #tf.summary.scalar('loss', self.loss)
        #self.accuracy = tf.reduce_mean(tf.metrics.accuracy(self.layer_1_d, self.target_y))
        #tf.summary.scalar('acc', self.acc)
        #self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(self.loss)

In [81]:
def eval_on_val():
    x = validating_df.loc[:,state_features]
    l = sess.run([autoencoder.loss], feed_dict={autoencoder.X:x, autoencoder.batch_size:X_val.shape[0]})
    print ("validation loss is: ", l)

In [83]:
batch_size = 100
autoencoder = AutoEncoder()
#tf.summary.scalar('loss', autoencoder.loss)
# tf.summary.scalar('acc', autoencoder.acc)
#merged = tf.summary.merge_all()
saver = tf.train.Saver()

init = tf.global_variables_initializer()

with tf.Session() as sess:
    #writer = tf.summary.FileWriter('logs/sparse_auto', sess.graph)
    sess.run(init)
    # Training cycle
    for step in range(100000):
        #sum_loss = 0
        a = training_df.sample(n=batch_size)
        cur_state = a.loc[:, state_features]
        _, l = sess.run([autoencoder.optimizer, autoencoder.loss], feed_dict={autoencoder.X:cur_state, autoencoder.batch_size : batch_size})
        #sum_loss += l
        #writer.add_summary(summary, step)
        if step % 1000==0 and step > 0:
            print ("step is ", step )
            #print ("total loss is ", sum_loss)
            print ("total loss is ", l)
            sum_loss = 0
        if step % 20000 == 0 and step >0:
            b = training_df.sample(n=1)
            cur_state_trial = b.loc[:, state_features]
            target_y, reconstruction = sess.run([autoencoder.target_y,autoencoder.layer_1_d],feed_dict={autoencoder.X:cur_state_trial})
            print (reconstruction)
            print (target_y)
        if step % 20000 == 0 and step >0:
            eval_on_val()
    #save_path = saver.save(sess, "sparse_autoencoder_model/model.ckpt")
    #print ('model saved to ', save_path)
    train_states = train_df.loc[:,state_features]
    train_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:train_states})

    train_autoencode = pd.DataFrame(train_repr[0])
    train_autoencode['vaso_input'] = train_df['vaso_input']
    train_autoencode['iv_input'] = train_df['iv_input']
    train_autoencode['reward'] = train_df['reward']
    train_autoencode['icustayid'] = train_df['icustayid']

    test_states = test_df.loc[:,state_features]
    test_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:test_states})
    test_autoencode = pd.DataFrame(test_repr[0])
    test_autoencode['vaso_input'] = test_df['vaso_input']
    test_autoencode['iv_input'] = test_df['iv_input']
    test_autoencode['reward'] = test_df['reward']
    test_autoencode['icustayid'] = test_df['icustayid']

step is  1000
total loss is  0.085078
step is  2000
total loss is  0.0646194
step is  3000
total loss is  0.0567674
step is  4000
total loss is  0.0462294
step is  5000
total loss is  0.040888
step is  6000
total loss is  0.0319459
step is  7000
total loss is  0.026447
step is  8000
total loss is  0.0219661
step is  9000
total loss is  0.0204112
step is  10000
total loss is  0.0175478
step is  11000
total loss is  0.0139173
step is  12000
total loss is  0.0129454
step is  13000
total loss is  0.0110771
step is  14000
total loss is  0.0103279
step is  15000
total loss is  0.00935378
step is  16000
total loss is  0.00875104
step is  17000
total loss is  0.00764216
step is  18000
total loss is  0.0073516
step is  19000
total loss is  0.00607876
step is  20000
total loss is  0.00580827
[[ 0.37942141  0.52488047  0.05130713  0.6890443   0.61134148  0.23946089
   0.52510422  0.35399994  0.33327174  0.65161383  0.91108268  0.95349026
   0.1542872   0.25871527  0.58023483  0.64660823  0.417905

step is  86000
total loss is  0.000446034
step is  87000
total loss is  0.000503749
step is  88000
total loss is  0.000421096
step is  89000
total loss is  0.00040679
step is  90000
total loss is  0.00047484
step is  91000
total loss is  0.000416231
step is  92000
total loss is  0.000391085
step is  93000
total loss is  0.00047117
step is  94000
total loss is  0.000416687
step is  95000
total loss is  0.000461491
step is  96000
total loss is  0.000390976
step is  97000
total loss is  0.000415428
step is  98000
total loss is  0.000394105
step is  99000
total loss is  0.000388024


KeyError: 'vaso_input'